In [1]:
%load_ext autoreload
%autoreload 2
import logging

logging.getLogger('mat73').setLevel(logging.CRITICAL)
import os
import re
from scipy.io import loadmat
import mat73
logging.getLogger('mat73').setLevel(logging.CRITICAL)

import warnings

from collections import Counter, defaultdict
import pandas as pd
import numpy as np
from scipy.stats import f_oneway
from sklearn import linear_model
from statsmodels.formula.api import ols
import statsmodels.api as sms

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from utils import *
from connectivity_dynamics import *
from eibal import *
from plotpowerbal import get_involved_dict
matplotlib.use('TkAgg')

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
matplotlib.use('TkAgg')
plt.ion()

In [3]:
POW_DIR = "/mnt/ernie_main/Ghassan/ephys/data/ei_bal/"
SPES_DIR = "/mnt/ernie_main/Ghassan/ephys/data/spes/"
tst_spes = "Epat26_3mA_SPES_windowed_PSD_300_length_10_stride_DISTGAP9999mm.mat"
raw_spes_f = "/home/ghassan/Documents/Research/Ephys/Projects/SPES/thresholded_dists/Epat26_3mA_SPES_windowed_PSD_300_length_10_stride_DISTGAP9999mm.mat"

tst_spes_struct = load_mat(os.path.join(SPES_DIR, "Epat26", tst_spes))
tst_spes_struct['spes']['pat_spes'].keys()
tst_spes_struct = tst_spes_struct['spes']['pat_spes']
tst_spes_struct.keys()

raw_spes = load_mat(raw_spes_f)
raw_spes = raw_spes['pat_spes']
raw_spes.keys()

dict_keys(['currents', 'data_RBP_SUB', 'data_RBP_SUB_AllDists', 'data_RBP_Z', 'data_RBP_Z_AllDists', 'data_SUB', 'data_SUB_AllDists', 'data_Z', 'data_Z_AllDists', 'data_axis_labels_for_each_current', 'dist_threshes_mm', 'mean_stim_pretrain_PSD', 'patID_clean', 'psd_freqs', 'rbp_bands', 'response_labels', 'response_soz', 'std_stim_pretrain_PSD', 'stim_labels', 'stim_soz', 'time_win_ms'])

In [4]:
data_RBP_Z = raw_spes['data_RBP_Z']
data_AllDists = raw_spes["data_RBP_Z_AllDists"]

In [5]:
tst_spes_struct['data_RBP_Z_AllDists'].shape

(53, 103, 5)

In [6]:
stim_labels = [s[0] for s in tst_spes_struct['stim_labels']]
resp_labels = tst_spes_struct['response_labels']
resp_labels = [r[0] for r in resp_labels]
stim_inds = [i for i, e in enumerate(resp_labels) if e in stim_labels]
z_freq = tst_spes_struct['data_RBP_Z_AllDists']
symm_conn = z_freq[:,stim_inds,:]


In [16]:
def z_score_spes(spes_conn, z_score, windowed=False, dist_thresh=[]):
    """Handles directed connectivity z-scoring for spes"""
    z_spes_in = np.zeros(spes_conn.shape)
    z_spes_out = np.zeros(spes_conn.shape)

    #Assuming that last entry is the band
    # assume shape: n_stim x n_resp x n_ bands
    #TODO make modular for windowing in future
    
    for b in range(spes_conn.shape[-1]):
        match z_score.lower():
            case 'row/col':
                #May not need to loop if linear algebra was on 100 today
                #TODO check that this is z_scored change in bip before each pulse
                z_spes_in[:,:,b] = z_score_conn(spes_conn[:,:,b], direction='col')
                z_spes_out[:,:,b] = z_score_conn(spes_conn[:,:,b], direction='row') 
            case "":
                #seems dumb, like I'm treating this as a symmetric matrix
                # and I lost modularity
                z= z_score_conn(spes_conn[:,:,b])
                z_spes_in[:,:,b] = z 
                z_spes_out[:,:,b] = z
    return z_spes_in, z_spes_out

def stim_to_df(stim_struct, symmetric=False, z_score='', data_col ='data_RBP_Z_AllDists' ):
    patID = stim_struct['patID_clean']
    stim_labels = [s[0] for s in stim_struct['stim_labels']]
    resp_labels = [r[0] for r in stim_struct['response_labels']]
    z_freq = stim_struct[data_col]

    resp_soz = [map_label(resp) for resp in stim_struct['response_soz']]
    stim_soz = [map_label(stim) for stim in stim_struct['stim_soz']]
    # Make SPES connectivity symmetric 
    # May be useful if worried about imbalance between
    #  number of stim and response sites
    resp_dict = defaultdict(lambda : [])
    if symmetric:
        assert set(stim_labels).issubset(set(resp_labels)), "Some stim labels not in resp labels!"
        stim_inds = [i for i, e in enumerate(resp_labels) if e in stim_labels]
        resp_labels = [resp_labels[i] for i in stim_inds]
        z_freq = z_freq[:,stim_inds,:]
        resp_soz = [resp_soz[i] for i in stim_inds]

    # Normalization part of pipeline. 
    # handle z_scoring here
    z_spes_in, z_spes_out = z_score_spes(z_freq, z_score)

    #Build up channel level dataframe
    for i, stim_site in enumerate(stim_labels):
        for j, band in enumerate(BANDS[1:]):
            resp_dict["stim_bip"].extend([stim_site]*len(resp_labels))
            resp_dict['resp_bip'].extend(resp_labels)
            conn_mat = z_freq[i,:,j]
            z_in = z_spes_in[i,:,j]
            z_out = z_spes_out[i,:,j]
            resp_dict[f'raw_change'].extend(conn_mat)
            resp_dict[f'z_in'].extend(z_in)
            resp_dict[f'z_out'].extend(z_out)
            resp_dict['band'].extend([band]*len(resp_labels))
            resp_dict['resp_label'].extend(resp_soz)
            resp_dict['stim_label'].extend([stim_soz[i]]*len(resp_labels))
    stim_df = pd.DataFrame.from_dict(resp_dict)
    stim_df['patID'] = patID
    stim_df['stim_resp'] = stim_df.apply(lambda x: f"{x['stim_label']}_{x['resp_label']}", axis=1)
    return stim_df

def get_general_involved(pow_df):
    involved_dict = defaultdict(lambda : True)
    for event in pow_df.eventID.unique():
        pow_event_df = pow_df[pow_df.eventID == event]
        pow_event_dict = get_involved_dict(pow_event_df,band='beta')
        for key in pow_event_dict.keys():
            involved_dict[key] = involved_dict[key] and pow_event_dict[key]
    return involved_dict

def get_anat_map(pow_df):
    assert len(pow_df.patID.unique()) == 1, "Only works if one patient per dataframe!"
    return dict(zip(pow_df.bip, pow_df.anat_region))

def merge_pow_spes(pow_df, spes_df, **kwargs):

    involved_dict = get_general_involved(pow_df)
    anat_dict = get_anat_map(pow_df)
    spes_df['beta_involved'] = spes_df.stim_bip.apply(lambda x: involved_dict[x])
    spes_df['target_involved'] = spes_df.resp_bip.apply(lambda x: involved_dict[x])
    spes_df['stim_anat_region'] = spes_df.stim_bip.apply(lambda x: anat_dict[x])
    spes_df['resp_anat_region'] = spes_df.stim_bip.apply(lambda x: anat_dict[x])
    return spes_df

def load_merge_spes_subj(spes_fname, pow_fname, **kwargs):
    spes_struct = load_mat(spes_fname)
    stim_df = stim_to_df(spes_struct['spes']['pat_spes'], **kwargs)
    pow_df = pd.read_csv(pow_fname)
    spes_pow_df = merge_pow_spes(pow_df, stim_df, **kwargs)
    return spes_pow_df

def load_merge_spes(subjects,pow_dir, spes_dir, **kwargs):
    spes_dfs =[]
    for subject in subjects:
        pow_fname = os.path.join(pow_dir, f"power_bal_{subject}_centered.csv")
        spes_fname = os.path.join(spes_dir, subject, f"{subject}_3mA_SPES_windowed_PSD_300_length_10_stride_DISTGAP9999mm.mat")

        try:
            assert os.path.exists(pow_fname)
            assert os.path.exists(spes_fname)
        except AssertionError:
            logger.warning(f"Subject {subject} not found in spes and power!")
            logger.warning(f"Pow status: {os.path.exists(pow_fname)} true for {pow_fname}")
            logger.warning(f"SPES status: {os.path.exists(spes_fname)} for {spes_fname}")
            continue
        df = load_merge_spes_subj(spes_fname, pow_fname, **kwargs)
        spes_dfs.append(df)
    return pd.concat(spes_dfs)

def construct_ISH_df(spes_df):
    """Returns summary df given of net soz in out spes connectivity
    Given the flattened SPES dfs we've been making because we cowtow to seaborn's 
    formatting for data"""
    ISH_spes_dict = defaultdict(lambda : [])
    for pat in spes_df.patID.unique():
        pat_df = spes_df[spes_df.patID == pat]
        for band in pat_df.band.unique():
            band_df = pat_df[pat_df.band == band]
            #SOZ stats 
            # NOTE: don't forget that if you want avg outward connectivity, that we normalize against inward!

            soz_in = np.mean(band_df[band_df.resp_label == 'SOZ'].z_out)
            soz_out = np.mean(band_df[band_df.stim_label == 'SOZ'].z_in)
            soz_net = soz_in - soz_out
            #build dict for df
            ISH_spes_dict['spes_conn'].extend([soz_in,soz_out, soz_net])
        
            #pz stats
            pz_in = np.nanmean(band_df[band_df.resp_label == 'PZ'].z_out)
            pz_out = np.nanmean(band_df[band_df.stim_label == 'PZ'].z_in)
            pz_net = pz_in - pz_out
            #update dict
            ISH_spes_dict['spes_conn'].extend([pz_in,pz_out, pz_net])

            #niz stats
            niz_in = np.mean(band_df[band_df.resp_label == 'NIZ'].z_out)
            niz_out = np.mean(band_df[band_df.stim_label == 'NIZ'].z_in)
            niz_net = niz_in - niz_out
        
            ISH_spes_dict['spes_conn'].extend([niz_in,niz_out, niz_net])
        
            ISH_spes_dict['dir'].extend(['inwards','outwards','net']*3)
            ISH_spes_dict['band'].extend([band]*9)
            ISH_spes_dict['region'].extend(['SOZ']*3 +['PZ']*3 +['NIZ']*3)
            ISH_spes_dict['patID'].extend([pat]*9)
            
    ISH_spes_df = pd.DataFrame.from_dict(ISH_spes_dict)
    return ISH_spes_df

def construct_net_df(spes_df):
    """Returns a net in-out for SPES connectivity on the channel (bipole ) level
    NOTE: assumes that you have constructed a symmetric SPES connectivity matrix. 
    This is important because the code will assume that for every stim-resp pair, you can 
    invert the relationship to get the reciprocal connectivity statistics"""
    def get_net(row):
        stim = row['stim']
        resp = row['resp']

    net_dfs = []
    for pat in spes_df.patID.unique():
        pat_df = spes_df[spes_df.patID == pat]
        for band in pat_df.band.unique():
            band_df = pat_df[pat_df.band == band]
            band_df['net_spes'] = band_df.apply(get_net, axis=0)
            net_dfs.append(band_df)
    return pd.concat(net_dfs)

# plotting lib
def plot_spes(plot_df,fname, x='stim_resp', y='raw_change',hue=None):
    
    # rename[False] ='Low Beta Recruitment'
    # rename[True] = "High Beta Recruitment"
    # plot_df.beta_involved = plot_df.beta_involved.apply(lambda x : rename[x])
    fname = fname.replace('.pdf','')
    for band in plot_df.band.unique():
        plt.close()
        plt.figure(figsize=(11,5))
        ax= sns.violinplot(data=plot_df[plot_df.band==band],x=x,y=y, hue=hue)
        #TODO extend whiskers
        #TODO visualize hypothesis test 
        plt.title(f"SPES change in Relative {band} Power")
        plt.xlabel("Stim-Response Pairs")
        plt.ylabel(f"Change in {band} power")
        plt.legend()
        plt.savefig(f"{fname}_{band}.pdf",transparent = True)

def run_stats(stats_df,relationship="spes_conn~region", test_type=1):
    """
    run 2 way anova"""
    cols = ['patID', 'bip',  'win_sz_centered','beta_involved', "win_label", 'freq_band','region_involved',  "value", 'z_beta']

    #TODO make bands more modular in future
    #TODO make modular for regionwise
    regions =  ['nz_nz_True', 'nz_nz_False', 'nz_soz_True', 'nz_soz_False']
    for band in stats_df.band.unique():
        print(f"Running {band}\n")
        band_stats_df = stats_df[stats_df.band == band]
        anova1way_model = ols(relationship, data=band_stats_df).fit()
        stats_res = sms.stats.anova_lm(anova1way_model, type=test_type)
        print(f"Computed stats for {band} window. Results:\n{stats_res}\n")
    return

In [ ]:

spes_subjects = pd.read_csv("../data/spes_subj.csv",header=None)
spes_subjects = spes_subjects[0].values

In [10]:

#spes_simp_df = load_merge_spes(spes_subjects[0:1], POW_DIR, SPES_DIR, symmetric = False, data_col = 'data_RBP_Z_AllDists')

### Loading SPES structs from Matlab

In [17]:
spes_norm_df = load_merge_spes(spes_subjects, POW_DIR, SPES_DIR, symmetric =False, z_score='row/col', data_col = 'data_RBP_Z_AllDists')

/tmp/ipykernel_870257/3206268140.py:93: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pow_df = pd.read_csv(pow_fname)
/tmp/ipykernel_870257/3206268140.py:93: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pow_df = pd.read_csv(pow_fname)
/tmp/ipykernel_870257/3206268140.py:93: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pow_df = pd.read_csv(pow_fname)


In [18]:
spes_norm_df

,stim_bip,resp_bip,raw_change,z_in,z_out,band,resp_label,stim_label,patID,stim_resp,beta_involved,target_involved,stim_anat_region,resp_anat_region
0,LA1LA2,LTP1LTP2,0.481620,0.118055,0.646338,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
1,LA1LA2,LTP2LTP3,0.614473,0.211162,0.842190,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
2,LA1LA2,LTP4LTP5,-0.107871,-0.688021,-0.222688,theta,NIZ,NIZ,Epat26,NIZ_NIZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
3,LA1LA2,LTP5LTP6,1.043761,0.693640,1.475046,theta,NIZ,NIZ,Epat26,NIZ_NIZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
4,LA1LA2,LTP6LTP7,0.003060,-0.529587,-0.059155,theta,NIZ,NIZ,Epat26,NIZ_NIZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26995,RTF6RTF7,LAC5LAC6,1.343412,0.047975,0.062409,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True,False,ctx-rh-inferiortemporal,ctx-rh-inferiortemporal
26996,RTF6RTF7,LAC6LAC7,-0.326764,-0.304172,-0.272370,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True,False,ctx-rh-inferiortemporal,ctx-rh-inferiortemporal
26997,RTF6RTF7,LAC7LAC8,-0.090543,-0.130383,-0.225021,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True,False,ctx-rh-inferiortemporal,ctx-rh-inferiortemporal
26998,RTF6RTF7,LAC8LAC9,0.346250,0.179668,-0.137468,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True,False,ctx-rh-inferiortemporal,ctx-rh-inferiortemporal


In [26]:
spes_norm_df.groupby(['patID','band','stim_bip','stim_anat_region']).count()

resp_bip  raw_change  \
patID  band  stim_bip stim_anat_region                                       
Epat26 alpha LA1LA2   ctx-lh-supramarginal                 103         103   
             LA3LA4   ctx-lh-supramarginal                 103         103   
             LA5LA6   ctx-lh-supramarginal                 103         103   
             LAC1LAC2 ctx-lh-caudalanteriorcingulate       103         103   
             LAC3LAC4 ctx-lh-superiorfrontal               103         103   
...                                                        ...         ...   
Spat52 theta RH1RH2   Right-Hippocampus                    100         100   
             RH3RH4   Right-Hippocampus                    100         100   
             RH5RH6   Right-Hippocampus                    100         100   
             RTF2RTF3 ctx-lh-fusiform                      100         100   
             RTF6RTF7 ctx-rh-inferiortemporal              100         100   

                                                      z_in  z_out  resp_label  \
patID  band  stim_bip stim_anat_region                                          
Epat26 alpha LA1LA2   ctx-lh-supramarginal             103    103         103   
             LA3LA4   ctx-lh-supramarginal             103    103         103   
             LA5LA6   ctx-lh-supramarginal             103    103         103   
             LAC1LAC2 ctx-lh-caudalanteriorcingulate   103    103         103   
             LAC3LAC4 ctx-lh-superiorfrontal           103    103         103   
...                                                    ...    ...         ...   
Spat52 theta RH1RH2   Right-Hippocampus                100    100         100   
             RH3RH4   Right-Hippocampus                100    100         100   
             RH5RH6   Right-Hippocampus                100    100         100   
             RTF2RTF3 ctx-lh-fusiform                  100    100         100   
             RTF6RTF7 ctx-rh-inferiortemporal          100    100         100   

                                                      stim_label  stim_resp  \
patID  band  stim_bip stim_anat_region                                        
Epat26 alpha LA1LA2   ctx-lh-supramarginal                   103        103   
             LA3LA4   ctx-lh-supramarginal                   103        103   
             LA5LA6   ctx-lh-supramarginal                   103        103   
             LAC1LAC2 ctx-lh-caudalanteriorcingulate         103        103   
             LAC3LAC4 ctx-lh-superiorfrontal                 103        103   
...                                                          ...        ...   
Spat52 theta RH1RH2   Right-Hippocampus                      100        100   
             RH3RH4   Right-Hippocampus                      100        100   
             RH5RH6   Right-Hippocampus                      100        100   
             RTF2RTF3 ctx-lh-fusiform                        100        100   
             RTF6RTF7 ctx-rh-inferiortemporal                100        100   

                                                      beta_involved  \
patID  band  stim_bip stim_anat_region                                
Epat26 alpha LA1LA2   ctx-lh-supramarginal                      103   
             LA3LA4   ctx-lh-supramarginal                      103   
             LA5LA6   ctx-lh-supramarginal                      103   
             LAC1LAC2 ctx-lh-caudalanteriorcingulate            103   
             LAC3LAC4 ctx-lh-superiorfrontal                    103   
...                                                             ...   
Spat52 theta RH1RH2   Right-Hippocampus                         100   
             RH3RH4   Right-Hippocampus                         100   
             RH5RH6   Right-Hippocampus                         100   
             RTF2RTF3 ctx-lh-fusiform                           100   
             RTF6RTF7 ctx-rh-inferiortemporal                   100   

                                 

## Group level analysis


In [29]:
spes_norm_df

,stim_bip,resp_bip,raw_change,z_in,z_out,band,resp_label,stim_label,patID,stim_resp,beta_involved,target_involved,stim_anat_region,resp_anat_region
0,LA1LA2,LTP1LTP2,0.481620,0.118055,0.646338,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
1,LA1LA2,LTP2LTP3,0.614473,0.211162,0.842190,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
2,LA1LA2,LTP4LTP5,-0.107871,-0.688021,-0.222688,theta,NIZ,NIZ,Epat26,NIZ_NIZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
3,LA1LA2,LTP5LTP6,1.043761,0.693640,1.475046,theta,NIZ,NIZ,Epat26,NIZ_NIZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
4,LA1LA2,LTP6LTP7,0.003060,-0.529587,-0.059155,theta,NIZ,NIZ,Epat26,NIZ_NIZ,False,False,ctx-lh-supramarginal,ctx-lh-supramarginal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26995,RTF6RTF7,LAC5LAC6,1.343412,0.047975,0.062409,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True,False,ctx-rh-inferiortemporal,ctx-rh-inferiortemporal
26996,RTF6RTF7,LAC6LAC7,-0.326764,-0.304172,-0.272370,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True,False,ctx-rh-inferiortemporal,ctx-rh-inferiortemporal
26997,RTF6RTF7,LAC7LAC8,-0.090543,-0.130383,-0.225021,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True,False,ctx-rh-inferiortemporal,ctx-rh-inferiortemporal
26998,RTF6RTF7,LAC8LAC9,0.346250,0.179668,-0.137468,gamma_H,NIZ,NIZ,Spat52,NIZ_NIZ,True,False,ctx-rh-inferiortemporal,ctx-rh-inferiortemporal


In [30]:
soz_niz_df = spes_norm_df[spes_norm_df.stim_resp.isin(['SOZ_NIZ'])]
# adaptniz_soz_df = construct_ISH_df(adaptniz_soz_df)

In [186]:
# niz_df = spes_norm_df[spes_norm_df.stim_resp.isin(['SOZ_NIZ'])]
# susceptible_df = niz_df[niz_df.beta_involved == True]
# susceptible_df = susceptible_df[susceptible_df.target_involved==True]
# susceptible_df
# adaptniz_badniz_df = construct_ISH_df(adaptniz_badniz_df)
# adaptniz_badniz_df['relationship'] = 'NIZ_NIZ'

,stim_bip,resp_bip,raw_change,z_in,z_out,band,resp_label,stim_label,patID,stim_resp,beta_involved,target_involved
10121,RPV11RPV12,ROF5ROF6,-0.763621,-1.309184,-0.515255,theta,NIZ,SOZ,Epat30,SOZ_NIZ,True,True
10127,RPV11RPV12,RMF9RMF10,0.204295,0.278642,0.424636,theta,NIZ,SOZ,Epat30,SOZ_NIZ,True,True
10131,RPV11RPV12,RMC5RMC6,-0.085424,0.039145,0.143306,theta,NIZ,SOZ,Epat30,SOZ_NIZ,True,True
10140,RPV11RPV12,RPI9RPI10,0.081898,0.121990,0.305783,theta,NIZ,SOZ,Epat30,SOZ_NIZ,True,True
10141,RPV11RPV12,RPI11RPI12,0.047544,-0.021456,0.272424,theta,NIZ,SOZ,Epat30,SOZ_NIZ,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
11578,LTT8LTT9,LAH9LAH10,0.399512,0.127212,-0.090322,gamma_H,NIZ,SOZ,Spat52,SOZ_NIZ,True,True
11584,LTT8LTT9,RH13RH14,0.308846,-0.082445,-0.098444,gamma_H,NIZ,SOZ,Spat52,SOZ_NIZ,True,True
11588,LTT8LTT9,RA10RA11,1.386707,0.712537,-0.001894,gamma_H,NIZ,SOZ,Spat52,SOZ_NIZ,True,True
11589,LTT8LTT9,RA12RA13,-0.729816,-0.176217,-0.191482,gamma_H,NIZ,SOZ,Spat52,SOZ_NIZ,True,True


In [191]:
niz_df = spes_norm_df[spes_norm_df.stim_resp.isin(['PZ_NIZ'])]
adaptable_df = niz_df[niz_df.beta_involved == True]
adaptable_df = adaptable_df[adaptable_df.target_involved==False]
adaptable_df

,stim_bip,resp_bip,raw_change,z_in,z_out,band,resp_label,stim_label,patID,stim_resp,beta_involved,target_involved
5060,RAI5RAI6,ROF3ROF4,0.714295,0.894077,0.389613,theta,NIZ,PZ,Epat30,PZ_NIZ,True,False
5062,RAI5RAI6,ROF7ROF8,1.341766,1.814933,1.705670,theta,NIZ,PZ,Epat30,PZ_NIZ,True,False
5063,RAI5RAI6,ROF11ROF12,0.646211,0.835831,0.246813,theta,NIZ,PZ,Epat30,PZ_NIZ,True,False
5064,RAI5RAI6,ROF13ROF14,0.634230,0.697087,0.221684,theta,NIZ,PZ,Epat30,PZ_NIZ,True,False
5068,RAI5RAI6,RMF11RMF12,0.820129,1.255636,0.611587,theta,NIZ,PZ,Epat30,PZ_NIZ,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9976,PC3PC4,PIC1PIC2,-0.452862,-0.221565,-0.875233,gamma_H,NIZ,PZ,Spat50,PZ_NIZ,True,False
9985,PC3PC4,MI1MI2,-0.449226,-0.156752,-0.873448,gamma_H,NIZ,PZ,Spat50,PZ_NIZ,True,False
9986,PC3PC4,MI3MI4,0.246950,-0.047295,-0.531551,gamma_H,NIZ,PZ,Spat50,PZ_NIZ,True,False
9987,PC3PC4,MI5MI6,0.549694,-0.086216,-0.382871,gamma_H,NIZ,PZ,Spat50,PZ_NIZ,True,False


In [192]:
res_df1 = adaptable_df[['z_in', 'band','patID']].groupby(['band','patID']).mean().reset_index()
res_df1['relationship'] = 'PZ-adaptable'

# res_df2 = susceptible_df[['z_in', 'band','patID']].groupby(['band','patID']).mean().reset_index()
# res_df2['relationship'] = 'late (NIZ)'

res_df3 = early_df[['z_in', 'band','patID']].groupby(['band','patID']).mean().reset_index()
res_df3['relationship'] = 'PZ-never'
res_df = pd.concat([res_df1, res_df3]).reset_index()
res_df

,index,band,patID,z_in,relationship
0,0,alpha,Epat30,0.508814,PZ-adaptable
1,1,alpha,Epat31,-0.664330,PZ-adaptable
2,2,alpha,Epat34,-0.001383,PZ-adaptable
3,3,alpha,Epat35,0.072954,PZ-adaptable
4,4,alpha,Epat37,0.179200,PZ-adaptable
...,...,...,...,...,...
140,65,theta,Spat34,-0.065060,PZ-never
141,66,theta,Spat36,0.013999,PZ-never
142,67,theta,Spat37,0.111718,PZ-never
143,68,theta,Spat41,0.112494,PZ-never


In [193]:
plot_df = res_df
with sns.plotting_context("paper"):
    grid = sns.FacetGrid(plot_df, row='band',row_order=['theta','alpha','beta','gamma_l','gamma_H'],\
                             legend_out=True)
    ax = grid.map(sns.violinplot,  'relationship', 'z_in','relationship')
    # ax.set(ylim=(-0.5,.75))
    plt.savefig("../viz/spes/adaptable_SPES.pdf")

/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:718: UserWarning: Using the violinplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:723: UserWarning: Using the violinplot function without specifying `hue_order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

Setting a gradient palette using color= is deprecated and will be removed in v0.14.0. Set `palette='dark:#1f77b4'` for the same effect.

  func(*plot_args, **plot_kwargs)
/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

Setting a gradient palette using color= is deprecated and will be removed in v0.14.0. Set `palette='dark:#1f77b4'` for the same effect.

  func(*plot_args, **plo

In [195]:
for band in plot_df.band.unique():
    band_df = plot_df[plot_df.band == band]
    involved = band_df[band_df.relationship =='PZ-adaptable'].z_in
    not_involved = band_df[band_df.relationship =='PZ-never'].z_in
    # print(band_df.groupby("beta_involved").count())
    print(ttest_ind(involved, not_involved))

TtestResult(statistic=np.float64(-0.21181714444799007), pvalue=np.float64(0.8338405248461147), df=np.float64(27.0))
TtestResult(statistic=np.float64(0.9755241789900089), pvalue=np.float64(0.33796014558786114), df=np.float64(27.0))
TtestResult(statistic=np.float64(0.17026842130438005), pvalue=np.float64(0.8660685057843426), df=np.float64(27.0))
TtestResult(statistic=np.float64(0.07747539505396099), pvalue=np.float64(0.938816659059158), df=np.float64(27.0))
TtestResult(statistic=np.float64(-1.2441536841923506), pvalue=np.float64(0.22413601378311993), df=np.float64(27.0))


In [104]:
plot_df = pd.concat([adaptniz_soz_df, adaptniz_badniz_df]).reset_index()

In [105]:
plot_df.relationship.unique()

array(['NIZ_SOZ', 'NIZ_NIZ'], dtype=object)

In [108]:
plot_df = plot_df[plot_df.region != 'PZ']
with sns.plotting_context("paper"):
    grid = sns.FacetGrid(plot_df, row='band',row_order=['theta','alpha','beta','gamma_l','gamma_H'],\
                            col='dir',col_order=['inwards','outwards','net'], legend_out=True)
    ax = grid.map(sns.violinplot,  'region', 'spes_conn','relationship',palette=COLOR_MAP)
    # ax.set(ylim=(-0.5,.75))
    plt.savefig("../viz/spes/adaptable_SPES.pdf")

/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:718: UserWarning: Using the violinplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:723: UserWarning: Using the violinplot function without specifying `hue_order` is likely to produce an incorrect plot.
  warnings.warn(warning)


# Plotting

### ISH Recapitulate Results

In [16]:
## make ISH spes df spes_norm_df
## god I know this is inefficient but we're scientists using computers
## not computer scientists
## at least its clean and legible

ISH_spes_df = construct_ISH_df(spes_norm_df)

/tmp/ipykernel_731293/803124654.py:126: RuntimeWarning: Mean of empty slice
  pz_in = np.nanmean(band_df[band_df.resp_label == 'PZ'].z_out)
/tmp/ipykernel_731293/803124654.py:127: RuntimeWarning: Mean of empty slice
  pz_out = np.nanmean(band_df[band_df.stim_label == 'PZ'].z_in)


In [17]:
with sns.plotting_context("paper"):
    grid = sns.FacetGrid(ISH_spes_df, row='band',row_order=['theta','alpha','beta','gamma_l','gamma_H'],\
                            col='dir',col_order=['inwards','outwards','net'], legend_out=True)
    ax = grid.map(sns.violinplot,  'region', 'spes_conn','region',palette=COLOR_MAP)
    # ax.set(ylim=(-0.5,.75))
    plt.savefig("../viz/spes/ISH_ccep_symm.pdf")

/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:718: UserWarning: Using the violinplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:723: UserWarning: Using the violinplot function without specifying `hue_order` is likely to produce an incorrect plot.
  warnings.warn(warning)


In [19]:
ISH_spes_df

,spes_conn,dir,band,region,patID
0,-0.041004,inwards,theta,SOZ,Epat26
1,0.077334,outwards,theta,SOZ,Epat26
2,-0.118338,net,theta,SOZ,Epat26
3,0.128372,inwards,theta,PZ,Epat26
4,0.404805,outwards,theta,PZ,Epat26
...,...,...,...,...,...
940,0.226109,outwards,gamma_H,PZ,Spat52
941,-0.182621,net,gamma_H,PZ,Spat52
942,-0.008214,inwards,gamma_H,NIZ,Spat52
943,-0.005178,outwards,gamma_H,NIZ,Spat52


In [20]:
run_stats(ISH_spes_df[ISH_spes_df.dir =='net' ])

Running theta

Computed stats for theta window. Results:
            df    sum_sq   mean_sq         F    PR(>F)
region     2.0  0.280190  0.140095  7.585489  0.001185
Residual  58.0  1.071191  0.018469       NaN       NaN

Running alpha

Computed stats for alpha window. Results:
            df    sum_sq   mean_sq         F    PR(>F)
region     2.0  0.003452  0.001726  0.047311  0.953828
Residual  58.0  2.115738  0.036478       NaN       NaN

Running beta

Computed stats for beta window. Results:
            df    sum_sq   mean_sq         F    PR(>F)
region     2.0  0.403787  0.201894  4.705682  0.012769
Residual  58.0  2.488443  0.042904       NaN       NaN

Running gamma_l

Computed stats for gamma_l window. Results:
            df    sum_sq   mean_sq         F    PR(>F)
region     2.0  0.108243  0.054121  4.650806  0.013387
Residual  58.0  0.674946  0.011637       NaN       NaN

Running gamma_H

Computed stats for gamma_H window. Results:
            df    sum_sq   mean_sq         F 

In [40]:
spes_sub_df = spes_norm_df[spes_norm_df.stim_resp.isin(['NIZ_SOZ', 'SOZ_NIZ'])]
spes_sub_df

,stim_bip,resp_bip,raw_change,z_in,z_out,band,resp_label,stim_label,patID,stim_resp,beta_involved
0,LA1LA2,LTP1LTP2,0.481620,0.118055,0.716983,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False
3,LA1LA2,LAM1LAM2,0.356209,0.421417,0.538323,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False
14,LA1LA2,LAH1LAH2,-0.039205,-0.520691,-0.024986,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False
15,LA1LA2,LAH3LAH4,-0.769486,-1.274001,-1.065346,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False
19,LA1LA2,LIT1LIT2,-1.095991,-1.638857,-1.530487,theta,SOZ,NIZ,Epat26,NIZ_SOZ,False
...,...,...,...,...,...,...,...,...,...,...,...
14534,RTF6RTF7,LI3LI4,0.498731,-0.063420,-0.111028,gamma_H,SOZ,NIZ,Spat52,NIZ_SOZ,True
14535,RTF6RTF7,LI5LI6,3.642683,0.172882,0.445763,gamma_H,SOZ,NIZ,Spat52,NIZ_SOZ,True
14542,RTF6RTF7,LTT8LTT9,-0.208810,-0.219595,-0.236333,gamma_H,SOZ,NIZ,Spat52,NIZ_SOZ,True
14571,RTF6RTF7,LTF1LTF2,-0.060672,-0.173122,-0.210098,gamma_H,SOZ,NIZ,Spat52,NIZ_SOZ,True


In [45]:
adaptive_spes_df = construct_ISH_df(spes_sub_df[spes_sub_df.beta_involved==False])

adaptive_spes_df['beta_involved'] = False
seizing_spes_df = construct_ISH_df(spes_sub_df[spes_sub_df.beta_involved==True])
seizing_spes_df['beta_involved'] = True
stim_df = pd.concat([adaptive_spes_df, seizing_spes_df]).reset_index()
stim_df['region_involved'] = stim_df.apply(lambda x: f"{x['region']}_{x['beta_involved']}", axis=1)
stim_df = stim_df.dropna()

/tmp/ipykernel_731293/1216281513.py:126: RuntimeWarning: Mean of empty slice
  pz_in = np.nanmean(band_df[band_df.resp_label == 'PZ'].z_out)
/tmp/ipykernel_731293/1216281513.py:127: RuntimeWarning: Mean of empty slice
  pz_out = np.nanmean(band_df[band_df.stim_label == 'PZ'].z_in)
/tmp/ipykernel_731293/1216281513.py:126: RuntimeWarning: Mean of empty slice
  pz_in = np.nanmean(band_df[band_df.resp_label == 'PZ'].z_out)
/tmp/ipykernel_731293/1216281513.py:127: RuntimeWarning: Mean of empty slice
  pz_out = np.nanmean(band_df[band_df.stim_label == 'PZ'].z_in)


In [61]:
stim_df

,index,spes_conn,dir,band,region,patID,beta_involved,region_involved
0,0,-0.040303,inwards,theta,SOZ,Epat26,False,SOZ_False
1,1,0.064346,outwards,theta,SOZ,Epat26,False,SOZ_False
2,2,-0.104649,net,theta,SOZ,Epat26,False,SOZ_False
6,6,0.001926,inwards,theta,NIZ,Epat26,False,NIZ_False
7,7,-0.030827,outwards,theta,NIZ,Epat26,False,NIZ_False
...,...,...,...,...,...,...,...,...
1747,802,-0.132358,outwards,gamma_H,SOZ,Spat52,True,SOZ_True
1748,803,0.082438,net,gamma_H,SOZ,Spat52,True,SOZ_True
1752,807,-0.128607,inwards,gamma_H,NIZ,Spat52,True,NIZ_True
1753,808,-0.097211,outwards,gamma_H,NIZ,Spat52,True,NIZ_True


In [62]:
region = "NIZ"
plot_df = stim_df[stim_df.region == region].dropna()
# plot_df = stim_df
with sns.plotting_context("paper"):
    grid = sns.FacetGrid(plot_df, row='band',row_order=['theta','alpha','beta','gamma_l','gamma_H'],\
                            col='dir',col_order=['inwards','outwards','net'], legend_out=True)
    ax = grid.map(sns.violinplot,  'region_involved', 'spes_conn','region_involved',palette=COLOR_MAP)
    plt.suptitle("NIZ SPES activity vs Beta Involvement")
    # ax.set(ylim=(-0.5,.75))
    plt.savefig(f"../viz/spes/ISH_beta_NIZ_spes.pdf")

/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:718: UserWarning: Using the violinplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/ghassan/miniconda3/envs/ephys/lib/python3.11/site-packages/seaborn/axisgrid.py:723: UserWarning: Using the violinplot function without specifying `hue_order` is likely to produce an incorrect plot.
  warnings.warn(warning)


plot_df[plot_df.dir =='net']

In [63]:
plot_df

,index,spes_conn,dir,band,region,patID,beta_involved,region_involved
6,6,0.001926,inwards,theta,NIZ,Epat26,False,NIZ_False
7,7,-0.030827,outwards,theta,NIZ,Epat26,False,NIZ_False
8,8,0.032753,net,theta,NIZ,Epat26,False,NIZ_False
15,15,0.038363,inwards,alpha,NIZ,Epat26,False,NIZ_False
16,16,0.047754,outwards,alpha,NIZ,Epat26,False,NIZ_False
...,...,...,...,...,...,...,...,...
1744,799,-0.007848,outwards,gamma_l,NIZ,Spat52,True,NIZ_True
1745,800,-0.056969,net,gamma_l,NIZ,Spat52,True,NIZ_True
1752,807,-0.128607,inwards,gamma_H,NIZ,Spat52,True,NIZ_True
1753,808,-0.097211,outwards,gamma_H,NIZ,Spat52,True,NIZ_True


In [60]:
stats_df

,index,spes_conn,dir,band,region,patID,beta_involved,region_involved


In [64]:
stats_df = plot_df[plot_df.dir =='outwards'].dropna()
run_stats(stats_df,relationship="spes_conn~beta_involved", test_type=1)

Running theta

Computed stats for theta window. Results:
                 df    sum_sq   mean_sq         F    PR(>F)
beta_involved   1.0  0.006740  0.006740  0.184226  0.670397
Residual       35.0  1.280502  0.036586       NaN       NaN

Running alpha

Computed stats for alpha window. Results:
                 df    sum_sq   mean_sq         F    PR(>F)
beta_involved   1.0  0.014736  0.014736  0.599966  0.443792
Residual       35.0  0.859633  0.024561       NaN       NaN

Running beta

Computed stats for beta window. Results:
                 df    sum_sq   mean_sq         F    PR(>F)
beta_involved   1.0  0.000569  0.000569  0.009487  0.922964
Residual       35.0  2.099588  0.059988       NaN       NaN

Running gamma_l

Computed stats for gamma_l window. Results:
                 df    sum_sq   mean_sq         F    PR(>F)
beta_involved   1.0  0.001361  0.001361  0.035906  0.850805
Residual       35.0  1.326593  0.037903       NaN       NaN

Running gamma_H

Computed stats for gamma_H wi

In [65]:
stats_df

,index,spes_conn,dir,band,region,patID,beta_involved,region_involved
7,7,-0.030827,outwards,theta,NIZ,Epat26,False,NIZ_False
16,16,0.047754,outwards,alpha,NIZ,Epat26,False,NIZ_False
25,25,0.084495,outwards,beta,NIZ,Epat26,False,NIZ_False
34,34,-0.021343,outwards,gamma_l,NIZ,Epat26,False,NIZ_False
43,43,-0.023444,outwards,gamma_H,NIZ,Epat26,False,NIZ_False
...,...,...,...,...,...,...,...,...
1717,772,0.218151,outwards,theta,NIZ,Spat52,True,NIZ_True
1726,781,0.356761,outwards,alpha,NIZ,Spat52,True,NIZ_True
1735,790,0.168560,outwards,beta,NIZ,Spat52,True,NIZ_True
1744,799,-0.007848,outwards,gamma_l,NIZ,Spat52,True,NIZ_True


In [66]:
for band in stats_df.band.unique():
    band_df = stats_df[stats_df.band == band]
    involved = band_df[band_df.beta_involved].spes_conn
    not_involved = band_df[~band_df.beta_involved].spes_conn
    # print(band_df.groupby("beta_involved").count())
    print(ttest_ind(involved, not_involved))

TtestResult(statistic=np.float64(0.4292152758079758), pvalue=np.float64(0.6703971924748539), df=np.float64(35.0))
TtestResult(statistic=np.float64(0.7745746673918394), pvalue=np.float64(0.4437924784923787), df=np.float64(35.0))
TtestResult(statistic=np.float64(-0.0974002702070175), pvalue=np.float64(0.9229641710403076), df=np.float64(35.0))
TtestResult(statistic=np.float64(0.1894877484066403), pvalue=np.float64(0.8508051900067669), df=np.float64(35.0))
TtestResult(statistic=np.float64(0.25613230274947235), pvalue=np.float64(0.7993481648894568), df=np.float64(35.0))


In [35]:
plot_df = spes_norm_df[spes_norm_df.stim_label=='NIZ']
plot_df = plot_df[plot_df.resp_label == 'SOZ']
with sns.plotting_context("paper"):
    grid = sns.FacetGrid(plot_df, row='band', row_order=['theta','alpha','beta','gamma_l','gamma_H'],
                         col='beta_involved', col_order=[True, False], legend_out=True)
    ax = grid.map(sns.kdeplot,"z_in")
    plt.suptitle("Stim region involved vs not and SPES")
    plt.savefig('../viz/spes/ISH_alldists.pdf')

In [102]:
keep_rel =['SOZ_NIZ', 'NIZ_SOZ' ]

plot_df = spes_norm_df[spes_norm_df.stim_resp.isin(keep_rel)]
plot_spes(plot_df, fname="../viz/spes/ISH_z_in_SPES_plot", y='z_in', x='stim_resp')
plot_spes(plot_df, fname="../viz/spes/ISH_z_out_SPES_plot", y='z_out', x='stim_resp')

# for band in plot_df.band.unique():
#     plt.close()
#     plt.figure(figsize=(11,5))
#     ax= sns.violinplot(data=plot_df[plot_df.band==band], x='stim_resp', y='net_change_z',, palette=COLOR_MAP)
#     #TODO extend whiskers
#     #TODO visualize hypothesis test 
#     plt.title(f"SPES change in Relative {band} Power")
#     plt.xlabel("Stim-Response Pairs")
#     plt.ylabel(f"Change in {band} power")
#     plt.legend()
#     plt.savefig(f"../viz/spes/SPES_ISH_{band}_boxplot.pdf",transparent = True)

## not getting results that make sense with ISH



/tmp/ipykernel_453101/1968096171.py:124: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/tmp/ipykernel_453101/1968096171.py:124: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/tmp/ipykernel_453101/1968096171.py:124: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/tmp/ipykernel_453101/1968096171.py:124: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
/tmp/ipykernel_453101/1968096171.py:124: UserWarning: No artists with labels found to put in leg

### Examine by Beta Score (seizure involvement proxy)

In [95]:
# plot_df = spes_summary_df[spes_summary_df.stim_resp.isin(keep_rel)]
plot_df = spes_norm_df
keep_rel =['NIZ_NIZ', 'NIZ_PZ', 'NIZ_SOZ' ]
plot_df = plot_df[plot_df.stim_resp.isin(keep_rel)]
# rename = labels={False:"same", True:"same"}
# plot_df.beta_involved = plot_df.beta_involved.apply(lambda x : rename[x])
plot_spes(plot_df, '../viz/spes/single_pat_norm_spes.pdf')
## not getting results that make sense with ISH


In [57]:
COLOR_MAP

{'pz': '#D95319',
 'soz': '#A2142F',
 'nz': '#0072BD',
 'niz': '#0072BD',
 'pz_soz': '#D95319',
 'soz_soz': '#A2142F',
 'nz_soz': '#0072BD',
 'PZ': '#D95319',
 'SOZ': '#A2142F',
 'NZ': '#0072BD',
 'NIZ': '#0072BD',
 'PZ_SOZ': '#D95319',
 'SOZ_SOZ': '#A2142F',
 'NZ_SOZ': '#0072BD'}